1. Setup

Importovanje potrebnih biblioteka

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import string
import re
from wordcloud import WordCloud
from collections import Counter
import sys

Ucitavanje podataka

In [ ]:
df = pd.read_csv("../data/raw/synthetic_emotions.csv")

print("Shape:", df.shape) # daje broj redova i kolona
print('\n')

print(df.head()) # daje header i prvih 5 redova
print('\n')

print("Vrednosti koje nedostaju:\n ", df.isnull().sum())

2. Osnovna analiza sirovih podataka

Distribucija klasa

In [ ]:
plt.figure(figsize=(6,4))
sb.countplot(data=df, x='emotion', order=df['emotion'].value_counts().index)
plt.title("Distribucija emocija")
plt.xlabel("Emocija")
plt.ylabel("Broj primera")
plt.show()

Duzina tekstova

In [ ]:
# duzina u karakterima
df['text_length'] = df['text'].apply(len) #ovo ce da doda ovuu kolonu 

plt.figure(figsize=(6,4))
sb.histplot(df['text_length'], bins=50)
plt.title("Duzina tekstova po broju karaktera")
plt.xlabel("Broj karaktera")
plt.ylabel("Broj primera")
plt.show()

# duzina po broju reci
df['word_count'] = df['text'].apply(lambda x: len(x.split())) # ovo ce da doda ovuu kolonu 
plt.figure(figsize=(6,4))
sb.histplot(df['word_count'], bins=50)
plt.title("Duzina tekstova po broju reci")
plt.xlabel("Broj reci")
plt.ylabel("Broj primera")
plt.show()

Raspodela duzina po klasi

In [ ]:
plt.figure(figsize=(8,5))
sb.boxplot(data=df, x='emotion', y='text_length')
plt.title("Duzina tekstova po klasi")
plt.ylabel("Broj karaktera")
plt.show()

3. Provera kvaliteta podataka (duplikati, specijalni znakovi i brojevi)

In [ ]:
print("Dupliktai: ", df.duplicated().sum())

Provera specijalnih znakova

In [ ]:
def has_special_chars(text):
    return bool(re.search(r'[^a-zA-Z\s]', text))

df['has_special'] = df['text'].apply(has_special_chars)
print("\nImaju specijalne znakove:")
print(df['has_special'].value_counts())

svi redovi imaju barem jedan specijalni znak

Provera brojeva

In [ ]:
def has_numbers(text):
    return any(char.isdigit() for char in text)

df['has_numbers'] = df['text'].apply(has_numbers)
print("\nImaju brojeve:")
print(df['has_numbers'].value_counts())

samo 7 redova ima brojeve, sto nije znacajan brij i ne treba trositi vreme i resurse na njihovo uklanjanje

4. Word cloud vizuelizacija

Najcesce reci po klasi

In [ ]:
stopwords = set(['the', 'a', 'an', 'and', 'or', 'to', 'of', 'in', 'on', 'for', 'with', 'as', 'by', 'at', 'from', 'that', 'this', 'it', 'he', 'she', 'his', 'her', 
    'they', 'them', 'is', 'was', 'were', 'be', 'been', 'are', 'am', 'but', 'not', 'you', 'i', 'eyes', 'has', 'had', 'him', 'into', 'every'])

def clean_text(texts):
    words = " ".join(texts).lower().split()
    words = [w.translate(str.maketrans('', '', string.punctuation)) for w in words]
    words = [w for w in words if w and w not in stopwords]
    return " ".join(words)

for emotion in df['emotion'].unique():
    text = clean_text(df[df['emotion'] == emotion]['text'])
    
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    plt.figure(figsize=(8,4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f"Word cloud - {emotion}")
    plt.axis('off')
    plt.show()

5. Odredjivanje max_seq_len

In [ ]:
max_seq_len = int(df['word_count'].quantile(0.95))
print("max_seq_len: ", max_seq_len)

95% tekstova ima do 71 rec, pa je max_seq_len postavljen na 71
Duzi tekstovi ce biti skraceni, a kraci ce biti dopunjeni paddingom

6. Pozivanje skripte za preprocesiranje 

In [ ]:
sys.path.append('../')
from src.data_preprocessing import DatasetPreprocessor

print("Pokretanje skripte za preprocesiranje...")
preprocessor = DatasetPreprocessor(input_path="../data/raw/synthetic_emotions.csv", output_dir="../data/processed/")
df_cleaned = preprocessor.run_pipeline()

print('\n')
print("Prikaz prvih 5 redova preprocesiranog dataseta koji ide u model:")
display(df_cleaned.head())